In [64]:
DRUM_TYPES = {
    "CC": [
        27,  # -- china 1
        28,  # -- cymbal 1
        30,  # -- cymbal 3
        31,  # -- cymbal 4
        32,  # -- cymbal 5
        49,  # crash cymbal 1
        57,  # crash cymbal 2
        52,  # china cymbal
        55,  # splash cymbal
        51,  # ride cymbal
        59,  # ride cymbal 2
        53,  # ride bell
        "CC",  # crash (ddm-own)
        "c1",  # crash cymbal 1 (enst/drummer1,2)
        "cr1",  # crash cymbal 1 (enst/drummer2)
        "cr2",  # crash cymbal 1 (enst/drummer3)
        "cr5",  # crash cymbal 2 (enst/drummer3)
        "rc3",  # ride cymbal 1 (enst/drummer2)
        "rc2",  # ride cymbal 2 (enst/drummer1)
        "rc4",  # ride cymbal 2 (enst/drummer2)
        "c4",  # ride cymbal 2 (enst/drummer3)
        "ch5",  # china ride cymbal (enst/drummer2)
        "ch1",  # china ride cymbal (enst/drummer3)
        "spl2",  # splash cymbal (enst/drummer2)
    ],  # crash
    "OH": [
        23,  # -- open pedal
        24,  # -- open 1
        25,  # -- open 2
        26,  # -- open 3
        "ohh",
        46,  # hi-hat open
        "overheads",  # drum kit data
    ],  # hi-hat open
    "CH": [
        21,  # -- closed pedal
        22,  # -- closed Edge
        "chh",
        42,  # hi-hat cloased
        44,  # hi-hat pedal
        "HH",  # closed hi-hat (ddm-own)
    ],  # hi-hat closed
    "TT": [
        "mt",
        41,  # -- low tom 2
        43,  # -- low tom 1
        45,  # mid tom
        47,  # mid tom
        48,  # high-mid tom
        50,  # high tom
        58,  # -- vibra slap
        "toms",  # tom (drum kit data)
        "ltr",  # low-tom, hit on the rim (enst/drummer1)
        "lmt",  # mid-tom-2 (enst/drummer3)
        "lt",  # low-tom (enst)
        "lft",  # low-tom-2 (enst/drummer3)
    ],  # tom
    "SD": [
        "sd",
        37,  # rimshot
        38,  # snare drum
        40,  # electric snare drum
        "snare",  # snare drum (drum kit data)
        "SD",  # snare (ddm-own)
        "rs",  # rim shot
    ],  # snare
    "KK": [
        "bd",
        35,  # bass drum
        36,  # kick drum
        "kick",  # kick (drum kit data)
        "KD",  # kick (idmt)
        "KK",  # kick (ddm-own)
    ],  # kick
}
DRUM_MAP = {}
# Iterate over the DRUM_TYPES
for drum_type, values in DRUM_TYPES.items():
    # Iterate over the values for each drum_type
    for value in values:
        # Add the mapping to the new dictionary
        DRUM_MAP[value] = drum_type

In [65]:
DATA_ENST_NOT = (
    "accompaniment",
    "cowbell",
    "brushes",
    "no-snare",
    "mallets",
    "rods",
    "cross-sticks",
    "phrase_reggae",
    "salsa",
    # # -- drummer1
    # "096_solo_latin_hands",  # sd-
    # "108_minus-one_rock-60s_sticks",
    # "113_minus-one_charleston_sticks",
    # # -- drummer2
    # "108_solo_toms_sticks",  # sd-
    # "121_minus-one_charleston_sticks",  # cs
    # "124_minus-one_bossa_sticks",  # cs
    # "134_MIDI-minus-one_country-120_sticks",  # cs
    # "144_MIDI-minus-one_rock-113_sticks",  # cs
    # "150_MIDI-minus-one_soul-98_sticks",  # cs
    # # -- drummer3
    # "067_phrase_afro_simple_slow_sticks",  # cs (cross sticks)
    # "068_phrase_afro_simple_medium_sticks",  # cs
    # "090_phrase_shuffle-blues_complex_fast_sticks",  # sticks
    # "111_phrase_oriental_simple_fast_sticks",  # cs
    # "114_phrase_oriental_complex_fast_sticks",  # cs
    # "115_phrase_cha-cha_simple_slow_sticks",  # cs
    # "116_phrase_cha-cha_complex_slow_sticks",  # cb (cow bell)
    # "134_minus-one_bossa_sticks",  # cs
    # "140_MIDI-minus-one_bigband_sticks",  # cs
    # "160_MIDI-minus-one_soul-98_sticks",  # cs
)  # ENST dataset에서 제외할 데이터

In [66]:
from typing import List
import numpy as np


def _get_filtering_onsets(
    onsets: List[float], start: float, end: float
) -> List[float]:
    """
    start ~ end 초 까지의 onsets 배열 구하는 함수
    """
    filter_onset = np.array(onsets)
    if filter_onset.size == 0:
        return filter_onset

    end = filter_onset[-1] + 1 if end == None else end
    filter_onset = filter_onset[(filter_onset >= start) & (filter_onset < end)]
    result = filter_onset - start  # start 빼서 0초부터 시작으로 맞추기

    np.set_printoptions(precision=2)
    # print(f"-- ! {start} sec ~ {end} sec 파생한 onsets: ", result)
    return result
def _get_filtering_onsets_instrument(
    onsets, start: float, end: float
) :
    """
    start ~ end 초 까지의 instrument별 onsets 배열 구하는 함수
    """
    result = {drum: [] for drum in onsets}
    for drum, onsets_arr in onsets.items():
        # print(f"-- ! {drum} 악기의 onset ! --")
        result[drum] = _get_filtering_onsets(onsets_arr, start, end)
    return result
def get_onsets_instrument_from_txt(
        txt_path: str, start: float = 0, end: float = None, onset_dict={}
    ) :
        """
        -- TXT file에서 onset 읽어오기

        * onset_dict: {'CC':[], 'OH':[], 'CH':[], 'TT':[], 'SD':[], 'HH':[]}
        """
        # print("-- ! txt file location: ", txt_path)

        # Open the text file
        with open(txt_path, "r") as file:
            lines = file.readlines()

        # -- ex) 2.674 mt => 2.674 / mt->ST
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 2:
                onset_sec = float(parts[0])
                drum_type = parts[1]
                drum_type = DRUM_MAP[drum_type]

                if drum_type in onset_dict:
                    onset_dict[drum_type].append(onset_sec)

        onset_dict = _get_filtering_onsets_instrument(
            onset_dict, start, end
        )
        return onset_dict

In [67]:
import glob

e_gmd_data = glob.glob('../data/raw/ENST-drums-public-clean/**/*.txt', recursive=True)
drum_pitch = {}
for midi_file in e_gmd_data:
    if any(p in midi_file for p in DATA_ENST_NOT):
        continue
    try:
        midi_data = get_onsets_instrument_from_txt(midi_file)
    except Exception as err:
        print(midi_file)
        print(err)

../data/raw/ENST-drums-public-clean/drummer_1/annotation/096_solo_latin_hands.txt
'sd-'
../data/raw/ENST-drums-public-clean/drummer_1/annotation/108_minus-one_rock-60s_sticks.txt
'cb'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/108_solo_toms_sticks.txt
'sd-'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/121_minus-one_charleston_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/124_minus-one_bossa_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/134_MIDI-minus-one_country-120_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/144_MIDI-minus-one_rock-113_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_2/annotation/150_MIDI-minus-one_soul-98_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_3/annotation/067_phrase_afro_simple_slow_sticks.txt
'cs'
../data/raw/ENST-drums-public-clean/drummer_3/annotation/068_phrase_afro_simple_medium_sticks.txt
'cs'
../data/raw/ENST-drums-publi